1. 导入文件，查看原始数据
2. 将人口数据和各州简称数据进行合并
3. 将合并的数据中重复的abbreviation列进行删除 
4. 查看存在缺失数据的列 
5. 找到有哪些state/region使得state的值为NaN，进行去重操作 
6. 为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN 
7. 合并各州面积数据areas 
8. 我们会发现area(sq.mi)这一列有缺失数据，找出是哪些行,去除含有缺失数据的行 
9. 找出2010年的全民人口数据 
10.计算各州的人口密度 排序，并找出人口密度最高的五个州 df.sort_values()

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [63]:
peo = pd.read_csv('./data/state-population.csv')
peo.head(2)

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0


In [64]:
area = pd.read_csv('./data/state-areas.csv')
area.head(2)

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425


In [65]:
abbr = pd.read_csv('./data/state-abbrevs.csv')
abbr.head(2)

,state,abbreviation
0,Alabama,AL
1,Alaska,AK


In [66]:
# 将人口数据和各州简称数据进行合并
# 将合并的数据中重复的abbreviation列进行删除
abbr_peo = pd.merge(abbr, peo, left_on='abbreviation', right_on='state/region', how='outer')
abbr_peo.drop(labels='abbreviation', axis=1,inplace=True)
abbr_peo.head(2)

,state,state/region,ages,year,population
0,Alabama,AL,under18,2012,1117489.0
1,Alabama,AL,total,2012,4817528.0


In [67]:
# 查看存在缺失数据的列
abbr_peo.isnull().any(axis=0)

state            True
state/region    False
ages            False
year            False
population       True
dtype: bool

In [81]:
# 找到有哪些state/region使得state的值为NaN，进行去重操作
abbr_peo.loc[abbr_peo.state[abbr_peo.state.isnull()].index]['state/region'].unique()

array([], dtype=object)

In [78]:
# 为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN 
indexs = abbr_peo[abbr_peo['state/region'] == 'PR'].index
abbr_peo.loc[indexs, 'state'] = 'PUERTO RICO'

In [80]:
indexs = abbr_peo[abbr_peo['state/region'] == 'USA'].index
abbr_peo.loc[indexs, 'state'] = 'UNITED STATES'

In [84]:
# 合并各州面积数据areas 
abbr_peo_area = pd.merge(abbr_peo, area, how='outer')
abbr_peo_area.head(2)

,state,state/region,ages,year,population,area (sq. mi)
0,Alabama,AL,under18,2012.0,1117489.0,52423.0
1,Alabama,AL,total,2012.0,4817528.0,52423.0


In [93]:
# 我们会发现area(sq.mi)这一列有缺失数据，找出是哪些行,去除含有缺失数据的行 
indexs = abbr_peo_area[abbr_peo_area['area (sq. mi)'].isnull()].index
abbr_peo_area.drop(labels=indexs, axis=0, inplace=True)

In [126]:
# 找出2010年的全民人口数据
abbr_peo_area.query("ages == 'total'& year==2010").head(2)
abbr_peo_area.query("ages == 'total'and year==2010").head(2)

,state,state/region,ages,year,population,area (sq. mi),peo_dense
3,Alabama,AL,total,2010.0,4785570.0,52423.0,91.287603
91,Alaska,AK,total,2010.0,713868.0,656425.0,1.087509


In [114]:
# 计算各州的人口密度 排序，并找出人口密度最高的五个州 df.sort_values()
abbr_peo_area.peo_dense = abbr_peo_area.population / abbr_peo_area['area (sq. mi)']
abbr_peo_area.head(2)

,state,state/region,ages,year,population,area (sq. mi),peo_dense
0,Alabama,AL,under18,2012.0,1117489.0,52423.0,21.316769
1,Alabama,AL,total,2012.0,4817528.0,52423.0,91.897221


In [119]:
# 排序，并找出人口密度最高的五个州 df.sort_values()
abbr_peo_area.sort_values(by='peo_dense',axis=0, ascending=False).head(2)

,state,state/region,ages,year,population,area (sq. mi),peo_dense
391,District of Columbia,DC,total,2013.0,646449.0,68.0,9506.602941
385,District of Columbia,DC,total,2012.0,633427.0,68.0,9315.102941
